In [1]:
import geopandas as gpd
import rasterio
import rasterio.mask
import rasterio.transform
from rasterio.io import MemoryFile
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import netCDF4 as nc
import glob
from unidecode import unidecode


In [2]:
# --- ENCODING FIX FUNCTIONS ---

def reverse_corruption_and_unidecode(corrupted_name):
    """
    Reverse the latin1->utf8 corruption and then unidecode.
    """
    try:
        # Reverse the corruption: encode as latin1, decode as utf-8
        original_name = corrupted_name.encode("latin1").decode("utf-8")
        # Now unidecode the original name
        unidecoded_name = unidecode(original_name)
        return unidecoded_name
    except (UnicodeEncodeError, UnicodeDecodeError):
        # If that fails, try direct unidecode
        return unidecode(corrupted_name)


def fix_region_encoding(region_name):
    """
    Fix encoding issues in region names by applying reverse corruption + unidecode to ALL names.
    This ensures consistent encoding and removes special characters.
    """
    if not isinstance(region_name, str):
        return region_name
    
    # Apply encoding fix to ALL region names for consistency
    return reverse_corruption_and_unidecode(region_name)


# --- INPUTS & OUTPUTS ---
##Input here the ADM boundaries ADM1= province, ADM2= Municipality
# ADM boundaries
adm_levels = [
    ("ADM1", "../../workspace/boundaries/geoBoundaries-BRA-ADM1-all/geoBoundaries-BRA-ADM1.shp"),
    ("ADM2", "../../workspace/boundaries/geoBoundaries-BRA-ADM2-all/geoBoundaries-BRA-ADM2.shp")
]

# Hazards root directory (contains both TIF and NC files)
hazards_root_dir = "../../workspace/demo_inputs/hazards"

# TIF-specific configuration
tif_subdir = "flood"  # Subdirectory within hazards_root_dir for TIF files
scenario_codes = ["pc", "rcp26", "rcp85"]
scenario_labels = {
    "pc": "CurrentClimate",
    "rcp26": "RCP2.6",
    "rcp85": "RCP8.5"
}
return_periods = [10, 100, 1000]

# NC files will be auto-discovered recursively in hazards_root_dir
# Expected structure: hazards/{hazard_type}/{hazard_indicator}/{model_type}/{file}.nc

# Output directory for individual file results
output_dir = "../../workspace/precomputed_region_results"
os.makedirs(output_dir, exist_ok=True)

# Final combined output CSV path
output_csv = "../../workspace/precomputed_adm_hazards.csv"

adm_gdfs = {}
for adm_label, adm_path in adm_levels:
    print(f"Reading {adm_label} boundaries from {adm_path} ...")
    if not os.path.exists(adm_path):
        raise FileNotFoundError(f"Boundary file {adm_path} not found. Please ensure the ADM boundary shapefiles are available.")
    gdf = gpd.read_file(adm_path)
    # Standardize column names for code and name fields
    gdf.columns = [col.lower() for col in gdf.columns]
    # For ADM1, geoBoundaries usually uses shapeID or shapeName columns
    if adm_label == "ADM1":
        # Try to set 'adm_code' and 'adm_name'
        # Common geoBoundaries columns: shapeName (name), shapeID (code), sometimes ADM1_PCODE, etc.
        if "shapeid" in gdf.columns:
            gdf["adm_code"] = gdf["shapeid"]
        elif "adm1_pcode" in gdf.columns:
            gdf["adm_code"] = gdf["adm1_pcode"]
        else:
            raise ValueError(f"ADM1 boundary missing unique code column (e.g. shapeID, ADM1_PCODE): columns={gdf.columns}")

        if "shapename" in gdf.columns:
            gdf["adm_name"] = gdf["shapename"]
        elif "adm1_fr" in gdf.columns:
            gdf["adm_name"] = gdf["adm1_fr"]
        else:
            raise ValueError(f"ADM1 boundary missing name column (e.g. shapeName, ADM1_FR): columns={gdf.columns}")
    elif adm_label == "ADM2":
        # Try to set 'adm_code' and 'adm_name'
        if "shapeid" in gdf.columns:
            gdf["adm_code"] = gdf["shapeid"]
        elif "adm2_pcode" in gdf.columns:
            gdf["adm_code"] = gdf["adm2_pcode"]
        else:
            raise ValueError(f"ADM2 boundary missing unique code column (e.g. shapeID, ADM2_PCODE): columns={gdf.columns}")

        if "shapename" in gdf.columns:
            gdf["adm_name"] = gdf["shapename"]
        elif "adm2_fr" in gdf.columns:
            gdf["adm_name"] = gdf["adm2_fr"]
        else:
            raise ValueError(f"ADM2 boundary missing name column (e.g. shapeName, ADM2_FR): columns={gdf.columns}")
    
    # Set the region column for use in processing
    gdf["region"] = gdf["adm_name"]
    
    # Apply encoding fixes to region names
    print(f"  Applying encoding fixes to {adm_label} region names...")
    gdf["region"] = gdf["region"].apply(fix_region_encoding)
    
    # Show examples of fixes applied
    original_names = gdf["adm_name"].tolist()
    fixed_names = gdf["region"].tolist()
    fixes_applied = [(orig, fixed) for orig, fixed in zip(original_names, fixed_names) if orig != fixed]
    
    print(f"  ✓ Applied encoding fixes to all {len(gdf)} region names in {adm_label}")
    if fixes_applied:
        print("  Examples of changes:")
        for i, (orig, fixed) in enumerate(fixes_applied[:5]):
            print(f"    '{orig}' -> '{fixed}'")
        if len(fixes_applied) > 5:
            print(f"    ... and {len(fixes_applied) - 5} more")
    else:
        print("  ✓ All region names were already properly encoded")

    adm_gdfs[adm_label] = gdf
    print(f"  ✓ Loaded {adm_label}: {len(gdf)} regions")

Reading ADM1 boundaries from ../../workspace/boundaries/geoBoundaries-BRA-ADM1-all/geoBoundaries-BRA-ADM1.shp ...
  Applying encoding fixes to ADM1 region names...
  ✓ Applied encoding fixes to all 27 region names in ADM1
  ✓ All region names were already properly encoded
  ✓ Loaded ADM1: 27 regions
Reading ADM2 boundaries from ../../workspace/boundaries/geoBoundaries-BRA-ADM2-all/geoBoundaries-BRA-ADM2.shp ...
  Applying encoding fixes to ADM2 region names...
  ✓ Applied encoding fixes to all 5570 region names in ADM2
  Examples of changes:
    'EspigÃ£o D'Oeste' -> 'Espigao D'Oeste'
    'GuajarÃ¡-Mirim' -> 'Guajara-Mirim'
    'Ji-ParanÃ¡' -> 'Ji-Parana'
    'Nova BrasilÃ¢ndia D'Oeste' -> 'Nova Brasilandia D'Oeste'
    'Presidente MÃ©dici' -> 'Presidente Medici'
    ... and 2380 more
  ✓ Loaded ADM2: 5570 regions


In [3]:
# --- NC TO RASTER CONVERSION HELPER ---

def nc_slice_to_raster(nc_file, gwl_idx, rp_idx, ensemble_value='mean'):
    """
    Convert a NetCDF slice to a rasterio-compatible in-memory raster.
    
    Uses the same cell-center to cell-edge conversion logic as R code:
    - Extract lon/lat coordinates (cell centers)
    - Calculate resolution: (max - min) / (n - 1)
    - Extend extent by half-pixel: xmin = min(lon) - res/2, xmax = max(lon) + res/2
    
    Args:
        nc_file: Path to NetCDF file
        gwl_idx: Index for GWL dimension (0-based)
        rp_idx: Index for return_period dimension (0-based)
        ensemble_value: Value to filter for ensemble dimension (default: 'mean')
    
    Returns:
        tuple: (rasterio dataset in memory, metadata dict)
    """
    dataset = nc.Dataset(nc_file, 'r')
    
    # Find the main data variable (prefer one ending with '_max')
    var_names = [v for v in dataset.variables.keys() 
                 if v not in ['lat', 'lon', 'latitude', 'longitude', 'GWL', 'gwl', 'return_period', 'ensemble']]
    main_var = None
    for v in var_names:
        if v.endswith('_max'):
            main_var = v
            break
    if main_var is None and var_names:
        main_var = var_names[0]
    
    if main_var is None:
        dataset.close()
        return None, None
    
    var = dataset.variables[main_var]
    dim_names = [d for d in var.dimensions]
    
    # Identify dimension names (case-insensitive)
    lon_dim = next((d for d in dim_names if d.lower() in ['lon', 'longitude', 'x']), 'lon')
    lat_dim = next((d for d in dim_names if d.lower() in ['lat', 'latitude', 'y']), 'lat')
    ens_dim = next((d for d in dim_names if d.lower() == 'ensemble'), None)
    gwl_dim = next((d for d in dim_names if d.lower() in ['gwl', 'GWL']), None)
    
    # Remaining dimension is likely return_period
    rp_dim = next((d for d in dim_names 
                   if d not in [lon_dim, lat_dim, ens_dim, gwl_dim]), 'return_period')
    
    # Get coordinate values
    lon_vals = dataset.variables[lon_dim][:]
    lat_vals = dataset.variables[lat_dim][:]
    
    # Find ensemble index
    ens_idx = 0
    if ens_dim and ens_dim in dataset.variables:
        ens_vals = dataset.variables[ens_dim][:]
        if ens_vals.dtype.kind in ['U', 'S']:  # String type
            ens_list = [str(e) if isinstance(e, bytes) else e for e in ens_vals]
            if ensemble_value in ens_list:
                ens_idx = ens_list.index(ensemble_value)
    
    # Build slice indices
    slice_dict = {}
    for d in dim_names:
        if d == lon_dim or d == lat_dim:
            slice_dict[d] = slice(None)
        elif d == ens_dim:
            slice_dict[d] = ens_idx
        elif d == gwl_dim:
            slice_dict[d] = gwl_idx
        elif d == rp_dim:
            slice_dict[d] = rp_idx
        else:
            slice_dict[d] = 0
    
    # Extract 2D slice
    # Build proper indexing tuple based on dimension order
    idx_tuple = tuple(slice_dict[d] for d in dim_names)
    data_slice = var[idx_tuple]
    
    # Ensure 2D
    if data_slice.ndim > 2:
        data_slice = np.squeeze(data_slice)
    
    # Calculate resolution and extent (cell-center to cell-edge conversion)
    n_lon = len(lon_vals)
    n_lat = len(lat_vals)
    
    res_lon = (np.max(lon_vals) - np.min(lon_vals)) / (n_lon - 1) if n_lon > 1 else 1.0
    res_lat = (np.max(lat_vals) - np.min(lat_vals)) / (n_lat - 1) if n_lat > 1 else 1.0
    
    # Extend by half-pixel
    xmin = np.min(lon_vals) - res_lon / 2
    xmax = np.max(lon_vals) + res_lon / 2
    ymin = np.min(lat_vals) - res_lat / 2
    ymax = np.max(lat_vals) + res_lat / 2
    
    # Ensure correct orientation: if data is (lon, lat), transpose to (lat, lon)
    if data_slice.shape[0] == n_lon and data_slice.shape[1] == n_lat:
        data_slice = data_slice.T
    
    # Flip vertically so first row is max(lat)
    data_slice = np.flipud(data_slice)
    
    # Create transform
    transform = rasterio.transform.from_bounds(xmin, ymin, xmax, ymax, n_lon, n_lat)
    
    # Create in-memory raster
    memfile = MemoryFile()
    with memfile.open(
        driver='GTiff',
        height=n_lat,
        width=n_lon,
        count=1,
        dtype=data_slice.dtype,
        crs='EPSG:4326',
        transform=transform
    ) as dst:
        dst.write(data_slice, 1)
    
    # Get metadata for inventory
    gwl_vals = dataset.variables[gwl_dim][:] if gwl_dim and gwl_dim in dataset.variables else [gwl_idx]
    rp_vals = dataset.variables[rp_dim][:] if rp_dim in dataset.variables else [rp_idx]
    
    gwl_label = str(gwl_vals[gwl_idx]) if gwl_idx < len(gwl_vals) else f"idx{gwl_idx}"
    rp_label = str(rp_vals[rp_idx]) if rp_idx < len(rp_vals) else f"idx{rp_idx}"
    
    metadata = {
        'gwl': gwl_label,
        'return_period': rp_label,
        'main_var': main_var
    }
    
    dataset.close()
    
    # Return the memfile (stays open) and metadata
    return memfile, metadata


def discover_nc_files(hazards_dir):
    """
    Discover all NC files and extract their metadata from path structure.
    Expected: hazards/{hazard_type}/{hazard_indicator}/{model_type}/{file}.nc
    
    Returns:
        list of dicts with file path and parsed metadata
    """
    nc_files = glob.glob(os.path.join(hazards_dir, '**', '*.nc'), recursive=True)
    
    results = []
    for f in nc_files:
        # Parse path components
        parts = os.path.normpath(f).split(os.sep)
        
        if len(parts) >= 4:
            file_name = parts[-1]
            model_type = parts[-2]
            hazard_indicator = parts[-3]
            hazard_type = parts[-4]
        else:
            # Fallback
            file_name = os.path.basename(f)
            model_type = 'ensemble'
            hazard_indicator = 'indicator'
            hazard_type = 'unknown'
        
        results.append({
            'path': f,
            'hazard_type': hazard_type,
            'hazard_indicator': hazard_indicator,
            'model_type': model_type,
            'file_name': file_name
        })
    
    return results


In [4]:
# --- PROCESSING NC FILES ---
print("\n" + "="*60)
print("PART 2: Processing NetCDF files...")
print("="*60)

# Discover all NC files
nc_files_info = discover_nc_files(hazards_root_dir)

if not nc_files_info:
    print("⚠ No NetCDF files found. Skipping NC processing.")
else:
    print(f"Found {len(nc_files_info)} NetCDF file(s)")
    
    # Create progress bar for NC files
    pbar_nc = tqdm(total=len(nc_files_info), desc="Processing NC files", unit="file", position=0, leave=True, dynamic_ncols=True)
    
    for nc_info in nc_files_info:
        nc_path = nc_info['path']
        hazard_type = nc_info['hazard_type']
        hazard_indicator = nc_info['hazard_indicator']
        
        pbar_nc.set_description(f"Processing {hazard_type}/{hazard_indicator}")
        
        # Open NC file to discover dimensions
        try:
            dataset = nc.Dataset(nc_path, 'r')
            
            # Find dimensions
            dim_names = list(dataset.dimensions.keys())
            gwl_dim = next((d for d in dim_names if d.lower() in ['gwl', 'GWL']), None)
            rp_dim = next((d for d in dim_names if d.lower() in ['return_period']), None)
            
            # Get dimension sizes
            n_gwl = len(dataset.dimensions[gwl_dim]) if gwl_dim else 1
            n_rp = len(dataset.dimensions[rp_dim]) if rp_dim else 1
            
            # Get actual values for labeling
            gwl_vals = dataset.variables[gwl_dim][:] if gwl_dim and gwl_dim in dataset.variables else list(range(n_gwl))
            rp_vals = dataset.variables[rp_dim][:] if rp_dim and rp_dim in dataset.variables else list(range(n_rp))
            
            dataset.close()
            
            # Process each GWL x return_period combination
            for ig in range(n_gwl):
                for ir in range(n_rp):
                    gwl_label = str(gwl_vals[ig]) if ig < len(gwl_vals) else f"idx{ig}"
                    rp_label = str(rp_vals[ir]) if ir < len(rp_vals) else f"idx{ir}"
                    
                    # Check if output already exists
                    output_filename = f"{hazard_type}_{hazard_indicator}_gwl{gwl_label}_rp{rp_label}.csv"
                    output_path = os.path.join(output_dir, output_filename)
                    
                    if os.path.exists(output_path):
                        pbar_nc.write(f"⏭  Already processed: {output_filename}. Skipping.")
                        continue
                    
                    # NC files contain pre-computed statistics in ensemble dimension
                    # Extract each ensemble value separately instead of computing from mean
                    
                    # Detect available ensemble values
                    try:
                        test_ds = nc.Dataset(nc_path, 'r')
                        dim_names = list(test_ds.dimensions.keys())
                        ens_dim = next((d for d in dim_names if d.lower() == 'ensemble'), None)
                        
                        if ens_dim and ens_dim in test_ds.variables:
                            ens_vals = test_ds.variables[ens_dim][:]
                            if ens_vals.dtype.kind in ['U', 'S']:  # String type
                                ensemble_values = [str(e) if isinstance(e, bytes) else e for e in ens_vals]
                            else:
                                ensemble_values = ['mean']  # Fallback
                        else:
                            ensemble_values = ['mean']  # No ensemble dimension
                        
                        test_ds.close()
                    except Exception as e:
                        pbar_nc.write(f"⚠ Could not detect ensemble values: {e}")
                        ensemble_values = ['mean']
                    
                    # Map ensemble values to statistic columns
                    ensemble_to_stat = {
                        'mean': 'mean',
                        'median': 'median',
                        'p10': 'p10',
                        'p90': 'p90',
                        'p2.5': 'p2_5',
                        'p5': 'p5',
                        'p95': 'p95',
                        'p97.5': 'p97_5'
                    }
                    
                    # Results for this slice - one row per region
                    file_results = []
                    
                    # Process each ADM level
                    for adm_label, gdf in adm_gdfs.items():
                        # Loop through each polygon
                        for idx, row in gdf.iterrows():
                            geom = [row["geometry"]]
                            
                            # Initialize result for this region
                            result = {
                                "region": row["region"],
                                "adm_level": adm_label,
                                "scenario_code": gwl_label,
                                "scenario_name": gwl_label,
                                "hazard_return_period": rp_label,
                                "hazard_type": hazard_type,
                                "hazard_indicator": hazard_indicator,
                                "min": np.nan,
                                "max": np.nan,
                                "mean": np.nan,
                                "median": np.nan,
                                "p2_5": np.nan,
                                "p5": np.nan,
                                "p95": np.nan,
                                "p97_5": np.nan,
                                "n_obs": 0,
                                "max_x": np.nan,
                                "max_y": np.nan
                            }
                            
                            # Extract each ensemble value separately
                            for ens_value in ensemble_values:
                                # Convert NC slice to raster for this specific ensemble
                                memfile, metadata = nc_slice_to_raster(nc_path, ig, ir, ensemble_value=ens_value)
                                
                                if memfile is None:
                                    continue
                                
                                try:
                                    with memfile.open() as src:
                                        nodata_val = src.nodata
                                        
                                        # Mask and crop the raster
                                        out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
                                        data = out_image[0]
                                        
                                        # Create mask for valid data
                                        if nodata_val is not None:
                                            valid_mask = data != nodata_val
                                            valid_data = data[valid_mask]
                                        else:
                                            valid_mask = np.ones(data.shape, dtype=bool)
                                            valid_data = data
                                        
                                        # Handle masked arrays
                                        if np.ma.is_masked(valid_data):
                                            valid_data = valid_data.compressed()
                                        
                                        n_obs = valid_data.size
                                        
                                        if n_obs > 0:
                                            # Extract mean of this ensemble layer  
                                            extracted_value = float(np.mean(valid_data))
                                            
                                            # Map to appropriate statistic
                                            stat_name = ensemble_to_stat.get(ens_value, ens_value)
                                            if stat_name in result:
                                                result[stat_name] = extracted_value
                                            
                                            # Track n_obs from first ensemble
                                            if result["n_obs"] == 0:
                                                result["n_obs"] = n_obs
                                            
                                            # Get min/max from actual data
                                            if ens_value == 'mean':
                                                result["min"] = float(np.min(valid_data))
                                                result["max"] = float(np.max(valid_data))
                                                
                                                # Get max pixel location
                                                max_val = result["max"]
                                                indices = np.where((data == max_val) & valid_mask)
                                                if len(indices[0]) > 0:
                                                    row_idx = indices[0][0]
                                                    col_idx = indices[1][0]
                                                    max_coord = rasterio.transform.xy(out_transform, row_idx, col_idx)
                                                    result["max_x"] = max_coord[0]
                                                    result["max_y"] = max_coord[1]
                                        
                                except Exception as e:
                                    pbar_nc.write(f"⚠ Error extracting {ens_value} for {row['region']}: {e}")
                                finally:
                                    memfile.close()
                            
                            # Add result if we got data
                            if result["n_obs"] > 0:
                                file_results.append(result)
                    
                    # Save results for this slice
                    if file_results:
                        df_file = pd.DataFrame(file_results)
                        df_file.to_csv(output_path, index=False, encoding='utf-8-sig')
                        pbar_nc.write(f"  ✓ Saved {len(file_results)} records to {output_filename}")
            
        except Exception as e:
            pbar_nc.write(f"⚠ Error processing {os.path.basename(nc_path)}: {e}")
        
        # Update progress bar
        pbar_nc.update(1)
    
    # Close NC progress bar
    pbar_nc.close()
    
    print(f"\n✓ NC processing complete!")

print("\n" + "="*60)
print("All hazard processing complete!")
print("="*60)



PART 2: Processing NetCDF files...
Found 2 NetCDF file(s)


Processing demo_inputs/hazards:  50%|█████     | 1/2 [04:11<04:11, 251.96s/file]

⚠ Error processing GIRI_flood_depth_cube.nc: a6a84398-cac6-456e-ad40-69a4ebff734b.tif: Free disk space available is 17179869184 bytes, whereas 23328000000 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.


Processing Drought/SPI6:  50%|█████     | 1/2 [11:46<04:11, 251.96s/file]       

  ✓ Saved 5597 records to Drought_SPI6_gwlpresent_rp5.csv


...

  ✓ Saved 5597 records to Drought_SPI6_gwl3_rp25.csv


Processing Drought/SPI6:  50%|█████     | 1/2 [1:52:38<04:11, 251.96s/file]

  ✓ Saved 5597 records to Drought_SPI6_gwl3_rp50.csv


Processing Drought/SPI6: 100%|██████████| 2/2 [2:03:11<00:00, 3695.52s/file]

  ✓ Saved 5597 records to Drought_SPI6_gwl3_rp100.csv

✓ NC processing complete!

All hazard processing complete!


## Accessing the Data

You can now access the loaded datasets using the hierarchical structure:

```python
# Access specific dataset
ds = return_period_data['Compound']['FWI']['ensemble']['ensemble_return_periods.nc']

# Or iterate through all datasets
for category in return_period_data:
    for hazard in return_period_data[category]:
        for model_type in return_period_data[category][hazard]:
            for filename, dataset in return_period_data[category][hazard][model_type].items():
                # Do something with dataset
                pass
```


In [5]:

# --- HELPER FUNCTION ---

# Note: The fix_text function has been replaced with the more comprehensive
# fix_region_encoding function that handles latin1->utf8 corruption reversal
# and unidecode processing.


# --- LOAD ADM SHAPEFILES ---
print("Loading ADM boundary shapefiles...")
adm_gdfs = {}

for adm_label, adm_path in adm_levels:
    # Read the shapefile for this ADM level
    try:
        gdf = gpd.read_file(adm_path)
    except UnicodeDecodeError:
        gdf = gpd.read_file(adm_path, encoding='latin1')
    
    # Create a unique identifier if one doesn't exist
    if "unique_id" not in gdf.columns:
        gdf["unique_id"] = gdf.index + 1

    # Determine a region name field by checking common field names
    if "shapeName" in gdf.columns:
        gdf["region"] = gdf["shapeName"]
    elif "NAME" in gdf.columns:
        gdf["region"] = gdf["NAME"]
    elif "name" in gdf.columns:
        gdf["region"] = gdf["name"]
    else:
        print(f"Warning: No standard region name field found for {adm_label}. Available fields:")
        print(gdf.columns)
        gdf["region"] = gdf["unique_id"]

    # Apply comprehensive encoding fixes to region names
    gdf["region"] = gdf["region"].apply(fix_region_encoding)
    
    adm_gdfs[adm_label] = gdf
    print(f"  ✓ Loaded {adm_label}: {len(gdf)} regions")


# --- PROCESSING TIF FILES ---
print("\n" + "="*60)
print("PART 1: Processing TIF flood maps...")
print("="*60)

# Calculate total number of flood maps to process
total_flood_maps = len(scenario_codes) * len(return_periods)

# Create main progress bar for flood maps
pbar_files = tqdm(total=total_flood_maps, desc="Processing TIF files", unit="file", position=0, leave=True, dynamic_ncols=True)

# Construct TIF directory path
tif_maps_dir = os.path.join(hazards_root_dir, tif_subdir)

# Loop over each scenario and return period (i.e., each flood file)
for scenario_code in scenario_codes:
    scenario_label = scenario_labels[scenario_code]
    
    for rp in return_periods:
        # Construct the flood map path
        floodmap_filename = f"global_{scenario_code}_h{rp}glob.tif"
        floodmap_path = os.path.join(tif_maps_dir, floodmap_filename)
        
        # Update progress bar description
        pbar_files.set_description(f"Processing {scenario_label} - RP{rp}")
        
        # Check if output CSV already exists
        output_filename = f"flood_{scenario_code}_rp{rp}.csv"
        output_path = os.path.join(output_dir, output_filename)
        
        if os.path.exists(output_path):
            pbar_files.write(f"⏭  Already processed: {output_filename}. Skipping.")
            pbar_files.update(1)
            continue
        
        # If the flood map file does not exist, skip it
        if not os.path.exists(floodmap_path):
            pbar_files.write(f"⚠ Flood map not found: {floodmap_filename}. Skipping.")
            pbar_files.update(1)
            continue

        # Results for this specific flood file
        file_results = []
        
        # Open the floodmap raster
        with rasterio.open(floodmap_path) as src:
            nodata_val = src.nodata
            
            # Process each ADM level for this flood file
            for adm_label, gdf in adm_gdfs.items():
                
                # Loop through each polygon in the shapefile
                for idx, row in gdf.iterrows():
                    geom = [row["geometry"]]
                    
                    try:
                        # Mask and crop the raster to the polygon's extent
                        out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
                    except Exception as e:
                        pbar_files.write(f"⚠ Error processing region {row['region']} in {adm_label}: {e}")
                        continue

                    # Extract the data from the first band
                    data = out_image[0]

                    # Create a mask for valid data (exclude nodata values)
                    if nodata_val is not None:
                        valid_mask = data != nodata_val
                        valid_data = data[valid_mask]
                    else:
                        valid_mask = np.ones(data.shape, dtype=bool)
                        valid_data = data

                    # If valid_data is a masked array, extract the underlying data
                    if np.ma.is_masked(valid_data):
                        valid_data = valid_data.compressed()

                    # Determine the number of valid observations (pixels)
                    n_obs = valid_data.size

                    # Compute statistics
                    if n_obs == 0:
                        stats = {
                            "min": np.nan,
                            "max": np.nan,
                            "mean": np.nan,
                            "median": np.nan,
                            "p2_5": np.nan,
                            "p5": np.nan,
                            "p95": np.nan,
                            "p97_5": np.nan
                        }
                        max_coord = (np.nan, np.nan)
                    else:
                        stats = {
                            "min": float(np.min(valid_data)),
                            "max": float(np.max(valid_data)),
                            "mean": float(np.mean(valid_data)),
                            "median": float(np.percentile(valid_data, 50)),
                            "p2_5": float(np.percentile(valid_data, 2.5)),
                            "p5": float(np.percentile(valid_data, 5)),
                            "p95": float(np.percentile(valid_data, 95)),
                            "p97_5": float(np.percentile(valid_data, 97.5))
                        }

                        # Identify the pixel location of the maximum value
                        max_val = stats["max"]
                        indices = np.where((data == max_val) & valid_mask)
                        if len(indices[0]) > 0:
                            row_idx = indices[0][0]
                            col_idx = indices[1][0]
                            max_coord = rasterio.transform.xy(out_transform, row_idx, col_idx)
                        else:
                            max_coord = (np.nan, np.nan)

                    # Build the result r cord
                    result = {
                        "region": row["region"],
                        "adm_level": adm_label,
                        "scenario_code": scenario_code,
                        "scenario_name": scenario_label,
                        "hazard_return_period": rp,
                        "hazard_type": "flood",
                        "min": stats["min"],
                        "max": stats["max"],
                        "mean": stats["mean"],
                        "median": stats["median"],
                        "p2_5": stats["p2_5"],
                        "p5": stats["p5"],
                        "p95": stats["p95"],
                        "p97_5": stats["p97_5"],
                        "n_obs": n_obs,
                        "max_x": max_coord[0],
                        "max_y": max_coord[1]
                    }

                    file_results.append(result)
        
        # Save results for this file to individual CSV
        if file_results:
            df_file = pd.DataFrame(file_results)
            df_file.to_csv(output_path, index=False, encoding='utf-8-sig')
            pbar_files.write(f"  ✓ Saved {len(file_results)} records to {output_filename}")
        
        # Update file progress bar
        pbar_files.update(1)

# Close main progress bar
pbar_files.close()

print(f"\n✓ Processing complete!")



Loading ADM boundary shapefiles...
  ✓ Loaded ADM1: 27 regions
  ✓ Loaded ADM2: 5570 regions

PART 1: Processing TIF flood maps...


Processing RCP8.5 - RP1000: 100%|██████████| 9/9 [00:00<00:00, 58.46file/s]        

⏭  Already processed: flood_pc_rp10.csv. Skipping.
⏭  Already processed: flood_pc_rp100.csv. Skipping.
⏭  Already processed: flood_pc_rp1000.csv. Skipping.
⚠ Flood map not found: global_rcp26_h10glob.tif. Skipping.
⚠ Flood map not found: global_rcp26_h100glob.tif. Skipping.
⚠ Flood map not found: global_rcp26_h1000glob.tif. Skipping.
⏭  Already processed: flood_rcp85_rp10.csv. Skipping.
⏭  Already processed: flood_rcp85_rp100.csv. Skipping.
⏭  Already processed: flood_rcp85_rp1000.csv. Skipping.

✓ Processing complete!


In [9]:

# --- COMBINE AND EXPORT ---

print("\nCombining individual CSV files...")

# Get all CSV files from the output directory
csv_files = [f for f in os.listdir(output_dir) if f.endswith('.csv')]

if csv_files:
    # Read and combine all CSV files
    dfs = []
    for csv_file in tqdm(csv_files, desc="Reading CSV files", unit="file"):
        csv_path = os.path.join(output_dir, csv_file)
        df = pd.read_csv(csv_path)
        
        # Standardize column names before adding to list
        # Some files have 'hazard_return', others have 'hazard_return_period'
        # The R codebase expects 'hazard_return_period', so standardize to that
        if 'hazard_return' in df.columns and 'hazard_return_period' not in df.columns:
            df = df.rename(columns={'hazard_return': 'hazard_return_period'})
        
        dfs.append(df)
    
    # Concatenate all dataframes
    df_combined = pd.concat(dfs, ignore_index=True)
    
    # Apply encoding fixes to region names in the combined dataframe
    print("Applying encoding fixes to all region names in combined data...")
    original_regions = set(df_combined["region"].unique())
    df_combined["region"] = df_combined["region"].apply(fix_region_encoding)
    fixed_regions = set(df_combined["region"].unique())
    
    # Report on changes applied
    regions_changed = original_regions - fixed_regions
    print(f"✓ Applied encoding fixes to all {len(original_regions)} unique region names")
    if regions_changed:
        print("Examples of changes applied:")
        for i, region in enumerate(sorted(list(regions_changed))[:10]):
            fixed_region = fix_region_encoding(region)
            print(f"  '{region}' -> '{fixed_region}'")
        if len(regions_changed) > 10:
            print(f"  ... and {len(regions_changed) - 10} more")
    else:
        print("✓ All region names were already properly encoded")
    
    # Fill missing values with 0 ONLY for non-hazard_return_period columns
    # Preserve original hazard_return_period values
    hazard_return_period_values = df_combined['hazard_return_period'].copy()
    df_combined = df_combined.fillna(0)
    df_combined['hazard_return_period'] = hazard_return_period_values.fillna(0)  # Only fill actual NaN values
    
    # Save combined results
    df_combined.to_csv(output_csv, index=False, encoding='utf-8-sig')
    
    print(f"\n" + "="*60)
    print(f"✓ Combined results saved to: {output_csv}")
    print(f"✓ Individual file results saved to: {output_dir}/")
    print(f"✓ Total records: {len(df_combined)}")
    print(f"✓ CSV files combined: {len(csv_files)}")
    print(f"✓ Column structure standardized (hazard_return -> hazard_return_period)")
    print(f"✓ Hazard return period values preserved from original files")
    print(f"✓ Encoding fixes applied universally to all region names")
    print("="*60)
else:
    print("\n⚠ No CSV files found to combine. Check if any flood maps were processed.")



Combining individual CSV files...


Reading CSV files:   8%|▊         | 2/26 [00:00<00:01, 13.85file/s]

Reading CSV files: 100%|██████████| 26/26 [00:02<00:00, 12.40file/s]


Applying encoding fixes to all region names in combined data...
✓ Applied encoding fixes to all 7608 unique region names
Examples of changes applied:
  'Abadia de Goiás' -> 'Abadia de Goias'
  'Abadiânia' -> 'Abadiania'
  'Abaeté' -> 'Abaete'
  'Abaré' -> 'Abare'
  'Abatiá' -> 'Abatia'
  'Abaíra' -> 'Abaira'
  'Abreulândia' -> 'Abreulandia'
  'Acaraú' -> 'Acarau'
  'Acará' -> 'Acara'
  'Acauã' -> 'Acaua'
  ... and 2289 more

✓ Combined results saved to: ../../workspace/precomputed_adm_hazards.csv
✓ Individual file results saved to: ../../workspace/precomputed_region_results/
✓ Total records: 145522
✓ CSV files combined: 26
✓ Column structure standardized (hazard_return -> hazard_return_period)
✓ Hazard return period values preserved from original files
✓ Encoding fixes applied universally to all region names


In [8]:
output_dir


'../../workspace/precomputed_region_results'